# Check the GPU

In [2]:
!nvidia-smi

Wed Jun 10 12:10:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Import Libraries and Functions

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import os
import time
import datetime
import math
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(101)

# tensorflow imports
import tensorflow as tf
import tensorflow.keras.backend as K

# tf.keras imports
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Add, Softmax, GlobalMaxPool2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D, Input, ReLU, Lambda, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# Import Resnet50 and BatchNorm
from MResNet50 import ResNet50
from MBatchNorm import BatchNorm

# Image Augmenters
from albumentations import (Compose, HorizontalFlip, Cutout, PadIfNeeded, RandomCrop)

# One Cycle LR
from clr import *

# Setting Hyperparams

In [0]:
# Dictionary of hyperparameters
hparams = {
    'learning_rate' : 0.4,  # fixed learning rate
    'batch_size': 32,  # training batch size
    'momentum' : 0.9, # fixed momentum
    'n_epochs': 1,  # number of epochs to train
    'decay' : 1e-04, # learning rate*batch size
}

# Data Preprocessing

## Downloading Dataset

In [7]:
zip_file = tf.keras.utils.get_file(origin="https://s3.amazonaws.com/fast-ai-imageclas/imagenette-320.tgz",
                                   fname="imagenette-320.tgz", extract=True)
base_dir, _ = os.path.splitext(zip_file)

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

n_samples = sum([len(files) for r,d,files in os.walk(train_dir)])

341295104/341289752 [==============================] - 22s 0us/step


## Image Augmentation techniques



In [8]:
image_size = 224 # All images will be resized to 224x224

def CustomImageDataGen(input_img):
    seq = Compose([PadIfNeeded(232,232, p=1),
                   RandomCrop(224,224, p=1),
                   HorizontalFlip(p=0.5),
                   Cutout(num_holes=1, max_h_size=56, max_w_size=56, p=0.5)
                  ], p=1)
    output_img = seq(image = input_img)['image']
    return output_img

# Rescale all images by 1./255 and apply image augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                preprocessing_function=CustomImageDataGen)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=hparams['batch_size'],
                class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=hparams['batch_size'],
                class_mode='categorical')

Found 12894 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


# 1. Model with Modified Batch Normalization Stride 1 - 50 epochs

## Model Summary

In [9]:
IMG_SHAPE = (image_size, image_size, 3)
# Create the base model of Resnet50 with modified BatchNorm
model = ResNet50(include_top=False,
                weights=None,
                input_shape=IMG_SHAPE,
                stride=1)

model = tf.keras.Sequential([
  model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr= hparams['learning_rate'], momentum= hparams['momentum'], 
                              nesterov=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


## Model Training

In [0]:
# Write logs in csv
import csv
# field names  
fields = ['Model No.', 'Time(s)', 'Accuracy']  
    
# name of csv file  
filename = "/content/drive/My Drive/BatchNorm Research/30_Sess_logs_Resnet50.csv"

In [13]:
num_sessions = 1

history = []
all_time = []
all_accuracy = []

# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)   
    # writing the fields  
    csvwriter.writerow(fields)  

    # Run for 30 sessions
    for i in range(num_sessions):

      # train the model
      start = time.time()

      #Build and Compile
      #######################################################
      model = ResNet50(include_top=False,
                weights=None,
                input_shape=IMG_SHAPE,
                stride=1)

      model = tf.keras.Sequential([
        model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(10, activation='softmax')
      ])

      model.compile(loss='categorical_crossentropy',
                    optimizer=SGD(lr= hparams['learning_rate'], momentum= hparams['momentum'], 
                                    nesterov=True),
                    metrics=['accuracy'])
      ###########################################################
      print(f"Model-{i} compiled")
      #checkpointer
      checkpointer = ModelCheckpoint(filepath=f"bestmodel_MBN_{i}.h5", verbose=0, 
                                    save_best_only=True, monitor="val_accuracy")
      # one cycle lr
      lr_manager = OneCycleLR(max_lr= hparams['learning_rate'],
                              batch_size = hparams['batch_size'],
                              samples = n_samples,
                              epochs = hparams['n_epochs'],
                              maximum_momentum = hparams['momentum'],
                              verbose=False)

      # Fit the model
      print(f"Model-{i} is training...")
      history = model.fit(train_generator,
                          epochs = hparams['n_epochs'],
                          steps_per_epoch= np.ceil(n_samples/hparams['batch_size']),
                          validation_data=validation_generator,
                          callbacks=[lr_manager, checkpointer])
      
      history.append(hist)
      end = time.time()
      print ("Model took %0.2f seconds to train"%(end - start))
      all_time.append(end-start)

      # Evaluation
      bestmodel = load_model(f"bestmodel_MBN_{i}.h5", custom_objects={'BatchNorm': BatchNorm})
      score= bestmodel.evaluate(validation_generator, batch_size=hparams['batch_size'], verbose=0)
      print("Max Validation Accuracy:", score[1]*100, "%\n")
      all_accuracy.append(score[1])

      # Write logs in csv file
      csvwriter.writerow([f"Model-{i}", all_time[i], all_accuracy[i]])  

Model-0 compiled
Model-0 is training...
 29/403 [=>............................] - ETA: 4:06 - loss: 8.5726 - accuracy: 0.1088

KeyboardInterrupt: ignored